In [164]:
import glob, re
import numpy as np
import pandas as pd
from sklearn import *
from datetime import datetime
from xgboost import XGBRegressor

In [165]:
data = {
    'air_visit': pd.read_csv('C:/Users/Nivedha/Documents/Datasets/air_visit_data.csv'),
    'air_info': pd.read_csv('C:/Users/Nivedha/Documents/Datasets/air_store_info.csv'),
    'hpg_info': pd.read_csv('C:/Users/Nivedha/Documents/Datasets/hpg_store_info.csv'),
    'air_reserve': pd.read_csv('C:/Users/Nivedha/Documents/Datasets/air_reserve.csv'),
    'hpg_reserve': pd.read_csv('C:/Users/Nivedha/Documents/Datasets/hpg_reserve.csv'),
    'id': pd.read_csv('C:/Users/Nivedha/Documents/Datasets/store_id_relation.csv'),
    'date_info': pd.read_csv('C:/Users/Nivedha/Documents/Datasets/date_info.csv').rename(columns={'calendar_date':'visit_date'}),
    'sub': pd.read_csv('C:/Users/Nivedha/Documents/Datasets/sample_submission.csv')
    }

In [166]:
data['hpg_reserve'].head(10)

,hpg_store_id,visit_datetime,reserve_datetime,reserve_visitors
0,hpg_c63f6f42e088e50f,2016-01-01 11:00:00,2016-01-01 09:00:00,1
1,hpg_dac72789163a3f47,2016-01-01 13:00:00,2016-01-01 06:00:00,3
2,hpg_c8e24dcf51ca1eb5,2016-01-01 16:00:00,2016-01-01 14:00:00,2
3,hpg_24bb207e5fd49d4a,2016-01-01 17:00:00,2016-01-01 11:00:00,5
4,hpg_25291c542ebb3bc2,2016-01-01 17:00:00,2016-01-01 03:00:00,13
5,hpg_28bdf7a336ec6a7b,2016-01-01 17:00:00,2016-01-01 15:00:00,2
6,hpg_2a01a042bca04ad9,2016-01-01 17:00:00,2016-01-01 17:00:00,2
7,hpg_2a84dd9f4c140b82,2016-01-01 17:00:00,2016-01-01 15:00:00,2
8,hpg_2ad179871696901f,2016-01-01 17:00:00,2016-01-01 13:00:00,2
9,hpg_2c1d989eedb0ff83,2016-01-01 17:00:00,2016-01-01 15:00:00,6


In [167]:
data['hpg_reserve'] = pd.merge(data['hpg_reserve'], data['id'], how='inner', on=['hpg_store_id'])

for df in ['air_reserve','hpg_reserve']:
    data[df]['visit_datetime'] = pd.to_datetime(data[df]['visit_datetime'])
    data[df]['visit_datetime'] = data[df]['visit_datetime'].dt.date
    data[df]['reserve_datetime'] = pd.to_datetime(data[df]['reserve_datetime'])
    data[df]['reserve_datetime'] = data[df]['reserve_datetime'].dt.date
    data[df]['reserve_datetime_diff'] = data[df].apply(lambda r: (r['visit_datetime'] - r['reserve_datetime']).days, axis=1)
    merge1 = data[df].groupby(['air_store_id','visit_datetime'], as_index=False)[['reserve_datetime_diff', 'reserve_visitors']].sum().rename(columns={'visit_datetime':'visit_date', 'reserve_datetime_diff': 'rs1', 'reserve_visitors':'rv1'})
    merge2 = data[df].groupby(['air_store_id','visit_datetime'], as_index=False)[['reserve_datetime_diff', 'reserve_visitors']].mean().rename(columns={'visit_datetime':'visit_date', 'reserve_datetime_diff': 'rs2', 'reserve_visitors':'rv2'})
    data[df] = pd.merge(merge1, merge2, how='inner', on=['air_store_id','visit_date'])

In [168]:
data['hpg_reserve'].head(10)

,air_store_id,visit_date,rs1,rv1,rs2,rv2
0,air_00a91d42b08b08d9,2016-01-14,3,2,3.0,2.0
1,air_00a91d42b08b08d9,2016-01-15,6,4,6.0,4.0
2,air_00a91d42b08b08d9,2016-01-16,3,2,3.0,2.0
3,air_00a91d42b08b08d9,2016-01-22,3,2,3.0,2.0
4,air_00a91d42b08b08d9,2016-01-29,6,5,6.0,5.0
5,air_00a91d42b08b08d9,2016-02-05,16,2,16.0,2.0
6,air_00a91d42b08b08d9,2016-03-08,2,3,2.0,3.0
7,air_00a91d42b08b08d9,2016-04-04,4,1,4.0,1.0
8,air_00a91d42b08b08d9,2016-04-07,1,2,1.0,2.0
9,air_00a91d42b08b08d9,2016-04-08,13,7,6.5,3.5


In [169]:
data[df].head(10)

,air_store_id,visit_date,rs1,rv1,rs2,rv2
0,air_00a91d42b08b08d9,2016-01-14,3,2,3.0,2.0
1,air_00a91d42b08b08d9,2016-01-15,6,4,6.0,4.0
2,air_00a91d42b08b08d9,2016-01-16,3,2,3.0,2.0
3,air_00a91d42b08b08d9,2016-01-22,3,2,3.0,2.0
4,air_00a91d42b08b08d9,2016-01-29,6,5,6.0,5.0
5,air_00a91d42b08b08d9,2016-02-05,16,2,16.0,2.0
6,air_00a91d42b08b08d9,2016-03-08,2,3,2.0,3.0
7,air_00a91d42b08b08d9,2016-04-04,4,1,4.0,1.0
8,air_00a91d42b08b08d9,2016-04-07,1,2,1.0,2.0
9,air_00a91d42b08b08d9,2016-04-08,13,7,6.5,3.5


In [170]:
data['id'].head(10)

,air_store_id,hpg_store_id
0,air_63b13c56b7201bd9,hpg_4bc649e72e2a239a
1,air_a24bf50c3e90d583,hpg_c34b496d0305a809
2,air_c7f78b4f3cba33ff,hpg_cd8ae0d9bbd58ff9
3,air_947eb2cae4f3e8f2,hpg_de24ea49dc25d6b8
4,air_965b2e0cf4119003,hpg_653238a84804d8e7
5,air_a38f25e3399d1b25,hpg_50378da9ffb9b6cd
6,air_3c938075889fc059,hpg_349b1b92f98b175e
7,air_68301bcb11e2f389,hpg_2c09f3abb2220659
8,air_5f6fa1b897fe80d5,hpg_40aff6385800ebb1
9,air_00a91d42b08b08d9,hpg_fbe603376b5980fc


In [171]:
data['air_visit']['visit_date'] = pd.to_datetime(data['air_visit']['visit_date'])
data['air_visit']['dow'] = data['air_visit']['visit_date'].dt.dayofweek
data['air_visit']['year'] = data['air_visit']['visit_date'].dt.year
data['air_visit']['month'] = data['air_visit']['visit_date'].dt.month
data['air_visit']['visit_date'] = data['air_visit']['visit_date'].dt.date

In [172]:
data['sub']['visit_date'] = data['sub']['id'].map(lambda x: str(x).split('_')[2])
data['sub']['air_store_id'] = data['sub']['id'].map(lambda x: '_'.join(x.split('_')[:2]))
data['sub']['visit_date'] = pd.to_datetime(data['sub']['visit_date'])
data['sub']['dow'] = data['sub']['visit_date'].dt.dayofweek
data['sub']['year'] = data['sub']['visit_date'].dt.year
data['sub']['month'] = data['sub']['visit_date'].dt.month
data['sub']['visit_date'] = data['sub']['visit_date'].dt.date

In [173]:
unique_restaurant = data['sub']['air_store_id'].unique()
restaurants = pd.concat([pd.DataFrame({'air_store_id': unique_restaurant, 'dow': [i]*len(unique_restaurant)}) for i in range(7)], axis=0, ignore_index=True).reset_index(drop=True)

In [174]:
temp = data['air_visit'].groupby(['air_store_id','dow'], as_index=False)['visitors'].min().rename(columns={'visitors':'min_visitors'})
restaurants = pd.merge(restaurants, temp, how='left', on=['air_store_id','dow']) 
temp = data['air_visit'].groupby(['air_store_id','dow'], as_index=False)['visitors'].mean().rename(columns={'visitors':'mean_visitors'})
restaurants = pd.merge(restaurants, temp, how='left', on=['air_store_id','dow'])
temp = data['air_visit'].groupby(['air_store_id','dow'], as_index=False)['visitors'].median().rename(columns={'visitors':'median_visitors'})
restaurants = pd.merge(restaurants, temp, how='left', on=['air_store_id','dow'])
temp = data['air_visit'].groupby(['air_store_id','dow'], as_index=False)['visitors'].max().rename(columns={'visitors':'max_visitors'})
restaurants = pd.merge(restaurants, temp, how='left', on=['air_store_id','dow'])
temp = data['air_visit'].groupby(['air_store_id','dow'], as_index=False)['visitors'].count().rename(columns={'visitors':'count_observations'})
restaurants = pd.merge(restaurants, temp, how='left', on=['air_store_id','dow'])

In [175]:
restaurants = pd.merge(restaurants, data['air_info'], how='left', on=['air_store_id']) 
# NEW FEATURES FROM Georgii Vyshnia
restaurants['air_genre_name'] = restaurants['air_genre_name'].map(lambda x: str(str(x).replace('/',' ')))
restaurants['air_area_name'] = restaurants['air_area_name'].map(lambda x: str(str(x).replace('-',' ')))
lbl = preprocessing.LabelEncoder()
for i in range(10):
    restaurants['air_genre_name'+str(i)] = lbl.fit_transform(restaurants['air_genre_name'].map(lambda x: str(str(x).split(' ')[i]) if len(str(x).split(' '))>i else ''))
    restaurants['air_area_name'+str(i)] = lbl.fit_transform(restaurants['air_area_name'].map(lambda x: str(str(x).split(' ')[i]) if len(str(x).split(' '))>i else ''))
restaurants['air_genre_name'] = lbl.fit_transform(restaurants['air_genre_name'])
restaurants['air_area_name'] = lbl.fit_transform(restaurants['air_area_name'])

In [176]:
data['date_info']['visit_date'] = pd.to_datetime(data['date_info']['visit_date'])
data['date_info']['day_of_week'] = lbl.fit_transform(data['date_info']['day_of_week'])
data['date_info']['visit_date'] = data['date_info']['visit_date'].dt.date
train = pd.merge(data['air_visit'], data['date_info'], how='left', on=['visit_date']) 
test = pd.merge(data['sub'], data['date_info'], how='left', on=['visit_date']) 

train = pd.merge(train, restaurants, how='left', on=['air_store_id','dow']) 
test = pd.merge(test, restaurants, how='left', on=['air_store_id','dow'])

In [177]:
for df in ['air_reserve','hpg_reserve']:
    train = pd.merge(train, data[df], how='left', on=['air_store_id','visit_date']) 
    test = pd.merge(test, data[df], how='left', on=['air_store_id','visit_date'])

train['id'] = train.apply(lambda r: '_'.join([str(r['air_store_id']), str(r['visit_date'])]), axis=1)

train['total_reserv_sum'] = train['rv1_x'] + train['rv1_y']
train['total_reserv_mean'] = (train['rv2_x'] + train['rv2_y']) / 2
train['total_reserv_dt_diff_mean'] = (train['rs2_x'] + train['rs2_y']) / 2

test['total_reserv_sum'] = test['rv1_x'] + test['rv1_y']
test['total_reserv_mean'] = (test['rv2_x'] + test['rv2_y']) / 2
test['total_reserv_dt_diff_mean'] = (test['rs2_x'] + test['rs2_y']) / 2


In [178]:
train['date_int'] = train['visit_date'].apply(lambda x: x.strftime('%Y%m%d')).astype(int)
test['date_int'] = test['visit_date'].apply(lambda x: x.strftime('%Y%m%d')).astype(int)
train['var_max_lat'] = train['latitude'].max() - train['latitude']
train['var_max_long'] = train['longitude'].max() - train['longitude']
test['var_max_lat'] = test['latitude'].max() - test['latitude']
test['var_max_long'] = test['longitude'].max() - test['longitude']

In [179]:
train['lon_plus_lat'] = train['longitude'] + train['latitude'] 
test['lon_plus_lat'] = test['longitude'] + test['latitude']

lbl = preprocessing.LabelEncoder()
train['air_store_id2'] = lbl.fit_transform(train['air_store_id'])
test['air_store_id2'] = lbl.transform(test['air_store_id'])

col = [c for c in train if c not in ['id', 'air_store_id', 'visit_date','visitors']]
train = train.fillna(-1)
test = test.fillna(-1)

In [180]:
test['visitors'].head(10)

0    0
1    0
2    0
3    0
4    0
5    0
6    0
7    0
8    0
9    0
Name: visitors, dtype: int64

In [181]:
def RMSLE(y, pred):
    return metrics.mean_squared_error(y, pred)**0.5

In [182]:
model1 = ensemble.GradientBoostingRegressor(learning_rate=0.2, random_state=3, n_estimators=200, subsample=0.8, 
                      max_depth =10)


In [183]:
model1.fit(train[col], np.log1p(train['visitors'].values))



GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.2, loss='ls', max_depth=10, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=200, presort='auto', random_state=3,
             subsample=0.8, verbose=0, warm_start=False)

In [184]:
preds1 = model1.predict(train[col])

In [185]:
print('RMSE GradientBoostingRegressor: ', RMSLE(np.log1p(train['visitors'].values), preds1))

preds1 = model1.predict(test[col])



RMSE GradientBoostingRegressor:  0.34793105169484123


In [186]:
print('RMSE GradientBoostingRegressor: ', RMSLE(np.log1p(test['visitors'].values), preds1))

RMSE GradientBoostingRegressor:  2.752881287981128


In [187]:
model2 = neighbors.KNeighborsRegressor(n_jobs=-1, n_neighbors=4)

In [188]:
model2.fit(train[col], np.log1p(train['visitors'].values))

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=-1, n_neighbors=4, p=2,
          weights='uniform')

In [189]:
preds2 = model2.predict(train[col])

In [190]:
print('RMSE KNeighborsRegressor: ', RMSLE(np.log1p(train['visitors'].values), preds2))

RMSE KNeighborsRegressor:  0.41945171611410526


In [191]:
from sklearn.feature_selection import RFE

In [192]:
preds2 = model2.predict(test[col])

In [193]:
print(preds2)

[1.53389122 3.31345315 3.50675062 ... 1.39960549 1.39960549 1.39960549]


In [194]:
model3 = XGBRegressor(learning_rate=0.2, random_state=3, n_estimators=280, subsample=0.8, 
                      colsample_bytree=0.8, max_depth =12)

In [195]:
model3.fit(train[col], np.log1p(train['visitors'].values))

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, importance_type='gain',
       learning_rate=0.2, max_delta_step=0, max_depth=12,
       min_child_weight=1, missing=None, n_estimators=280, n_jobs=1,
       nthread=None, objective='reg:linear', random_state=3, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=0.8)

In [196]:
preds3 = model3.predict(train[col])

In [197]:
print('RMSE XGBRegressor: ', RMSLE(np.log1p(train['visitors'].values), preds3))

RMSE XGBRegressor:  0.40288770597836376


In [198]:
preds3 = model3.predict(test[col])

In [199]:
test['visitors'] = 0.3*preds1+0.3*preds2+0.4*preds3
test['visitors'] = np.expm1(test['visitors']).clip(lower=0.)


In [200]:
test['visitors'].head(10)

0     3.104401
1    22.520606
2    29.967686
3    26.371493
4    29.381655
5    38.060180
6    11.420190
7     3.104401
8    21.447386
9    30.054161
Name: visitors, dtype: float64

In [201]:
sub1 = test[['id','visitors']].copy()

In [202]:
print(sub1)

                                    id   visitors
0      air_00a91d42b08b08d9_2017-04-23   3.104401
1      air_00a91d42b08b08d9_2017-04-24  22.520606
2      air_00a91d42b08b08d9_2017-04-25  29.967686
3      air_00a91d42b08b08d9_2017-04-26  26.371493
4      air_00a91d42b08b08d9_2017-04-27  29.381655
5      air_00a91d42b08b08d9_2017-04-28  38.060180
6      air_00a91d42b08b08d9_2017-04-29  11.420190
7      air_00a91d42b08b08d9_2017-04-30   3.104401
8      air_00a91d42b08b08d9_2017-05-01  21.447386
9      air_00a91d42b08b08d9_2017-05-02  30.054161
10     air_00a91d42b08b08d9_2017-05-03  28.455899
11     air_00a91d42b08b08d9_2017-05-04  25.562437
12     air_00a91d42b08b08d9_2017-05-05  37.459890
13     air_00a91d42b08b08d9_2017-05-06  13.103645
14     air_00a91d42b08b08d9_2017-05-07   2.579532
15     air_00a91d42b08b08d9_2017-05-08  21.447386
16     air_00a91d42b08b08d9_2017-05-09  30.054161
17     air_00a91d42b08b08d9_2017-05-10  27.550144
18     air_00a91d42b08b08d9_2017-05-11  29.112681
